In [15]:
import json
Base_Skills=[] 
with open('data/jobs.json' , 'r' , encoding='UTF-8') as f :
    data=json.load(f)
    for job in data:
        if (job['skills']):
            List_skills=job['skills']
            for skill in List_skills:
                Base_Skills.append(skill)
    newdata={"SKILLS": Base_Skills}
    with open ('data/Skills_Data.json' , 'w' , encoding='UTF-8') as f:
        json.dump(newdata , f , ensure_ascii= False )


In [16]:
import spacy, re
from skweak import heuristics, gazetteers, generative, utils

tries=gazetteers.extract_json_data('data/Skills_Data.json')

Extracting data from data/Skills_Data.json
Populating trie for class SKILLS (number: 2394)


In [17]:
import spacy
nlp=spacy.load('en_core_web_md' , disable=['ner'])
gazetteer = gazetteers.GazetteerAnnotator("SKILLS", tries)
lf3= gazetteers.GazetteerAnnotator("SKILLS", tries)


In [19]:

def experience_detector (doc):
    expression=r'[0-9][+] years'
    for match in re.finditer(expression, doc.text):
        start, end = match.span()
        span = doc.char_span(start, end)
        if(span):
            yield span.start , span.end ,  "EXPERIENCE"
lf1 = heuristics.FunctionAnnotator("experience", experience_detector)
doc=lf1(nlp(' 2+ years of experience. 3+ years i worked with a lot of tools'))
doc.spans

{'experience': [2+ years, 3+ years]}

In [21]:
def Diploma_fun(doc):
    
    key_words=[ "phd","Bachelor", "Masters"  , "High School" , "MS/Ph.D."  , "BA" , "BS"  ,"College" , "Bachelors" ]
    for key in key_words:
                for match in re.finditer(key , doc.text , re.IGNORECASE):
                    start, end = match.span()
                    span = doc.char_span(start, end)
                    if(span):
                        yield (span.start , span.end ,  "DIPLOMA")
    
lf4 = heuristics.FunctionAnnotator("Diploma", Diploma_fun)

In [37]:
def Diploma_major_fun(doc):  
    key_words=['Computing Science','IA' ,'Mechanical Engineering', 'Electrical engineering' ,'Computer engineering' , "computer science"  , "finance" , "accounting" , "economics" , "business administration" , "Math" , "Acoustics" , "Physics" , "Computer Networks" , "technical discipline" , "Business" , "CS/EE" , "Telecommunications" , "Material Science" , "Robotics"]
    for key in key_words:
                for match in re.finditer(key , doc.text , re.IGNORECASE):
                    start, end = match.span()
                    span = doc.char_span(start, end)
                    if(span):
                        yield (span.start , span.end ,  "DIPLOMA_MAJOR")
    
lf2 = heuristics.FunctionAnnotator("Diploma_major", Diploma_major_fun)

In [ ]:
#create_corpus_annotated_to_train_the_model
docs=[]
with open('Corpus.txt' , 'r') as f :
    data=f.readlines()
    for text in data:
        if (len(text) !=1):
            newtext=str(text)
            doc=nlp(newtext)
            doc=lf1(lf2(lf3(lf4(doc))))
            print(doc.spans)
            docs.append(doc)



In [39]:
from skweak import aggregation
model = aggregation.HMM("hmm", ["DIPLOMA", "DIPLOMA_MAJOR" , "EXPERIENCE" , "SKILLS"])

In [ ]:
docs = model.fit_and_aggregate(docs)
utils.display_entities(docs[16], "hmm", add_tooltip=True) 

In [27]:
for doc in docs:
    doc.ents = doc.spans["hmm"]
utils.docbin_writer(docs, "reuters_small.spacy")

Write to reuters_small.spacy...done


In [455]:
#Train_The_Model
#577 itérations
!python -m spacy train config.cfg --output ./UBIAI-Model --paths.train train.spacy --paths.dev train.spacy

^C


In [150]:
TRAIN_DATA=[]
with open('annotation.json' , 'r' , encoding='UTF-8') as f:
    data=json.load(f)
    for d in data:
        entities=[]
        TEXT=d['document']
        annotations=d['annotation']
        for annotation in  annotations:
            new=(annotation['start'] , annotation['end'] , annotation['label'])
            entities.append(new)
        training=(TEXT ,{"entities": entities})
        TRAIN_DATA.append(training)



In [153]:
import spacy
from spacy.training import Example
from spacy.tokens import DocBin
nlp = spacy.blank("en")
db = DocBin()

for text, annotations in TRAIN_DATA:
    example = Example.from_dict(nlp.make_doc(text), annotations)
    db.add(example.reference)

db.to_disk("train.spacy")

Model _ Evaluation

In [28]:
import spacy
nlp=spacy.load('output/model-best')


In [114]:
Test_data=[]
with open ('Test_studio.json' , 'r' , encoding='UTF-8') as f:
    data=json.load(f)
for d in data:
    text=d['value']
    entities=d['lbl']
    format_entities=[]
    for entity in entities:
        new=[entity['start'] , entity['end'] , entity['labels'][0]]
        format_entities.append(new)
    
    new=(text ,{"entities": format_entities})
    Test_data.append(new)

In [131]:
#Format Test.json to Test.spacy
import spacy
from spacy.training import Example
from spacy.tokens import DocBin
nlp = spacy.blank("en")
db = DocBin()

for text, annotations in Test_data:
    example = Example.from_dict(nlp.make_doc(text), annotations)
    db.add(example.reference)

db.to_disk("Test.spacy")

In [178]:
!python -m spacy evaluate UBIAI-Model/model-best/ UBIAI_TEST.spacy

ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   75.64 
NER R   80.82 
NER F   78.15 
SPEED   3833  


=============================== NER (per type) ===============================

                     P        R        F
EXPERIENCE      100.00   100.00   100.00
SKILLS           67.24    73.58    70.27
DIPLOMA         100.00   100.00   100.00
DIPLOMA_MAJOR   100.00   100.00   100.00



In [179]:
!python -m spacy evaluate output/model-best/ UBIAI_TEST.spacy

ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   57.89 
NER R   30.14 
NER F   39.64 
SPEED   4362  


=============================== NER (per type) ===============================

                     P       R       F
EXPERIENCE      100.00   80.00   88.89
SKILLS           36.00   16.98   23.08
DIPLOMA         100.00   33.33   50.00
DIPLOMA_MAJOR   100.00   75.00   85.71

